In [2]:
import pandas as pd

In [4]:
df = pd.read_csv('AirlineData.csv')

In [6]:
df.head()

,Airline,Source,Destination,Number of Stops,Class,Date,Total_Stopover_Time,price in CAD,days_left,Departure_24hr,Arrival_24hr,Arrival_Day_Offset
0,Etihad Airways,Toronto Pearson Intl,Bengaluru Intl,1,Economy Class,2024-06-02,130,2340,1,22:10,3:05,2
1,Delta,Toronto Pearson Intl,Bengaluru Intl,1,Economy Class,2024-06-02,1335,1347,1,22:40,23:15,2
2,Multiple Airlines,Toronto Pearson Intl,Bengaluru Intl,2,Economy Class,2024-06-02,420,1934,1,21:20,7:45,2
3,Multiple Airlines,Toronto Pearson Intl,Bengaluru Intl,2,Economy Class,2024-06-02,380,2291,1,21:20,6:50,2
4,Multiple Airlines,Toronto Pearson Intl,Bengaluru Intl,2,Economy Class,2024-06-02,175,2661,1,23:40,6:25,2


In [10]:
df.shape

(824019, 12)

In [ ]:
pip install tensorflow 

In [ ]:
pip install torch

In [12]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input

# Load and preprocess data
df = pd.read_csv('AirlineData.csv')  # Update the path if needed

# Preprocessing steps
df['Date'] = pd.to_datetime(df['Date'])
df['Departure_24hr'] = pd.to_datetime(df['Departure_24hr'], format='%H:%M').dt.time
df['Arrival_24hr'] = pd.to_datetime(df['Arrival_24hr'], format='%H:%M').dt.time

df['Day_of_Week'] = df['Date'].dt.dayofweek
df['Month'] = df['Date'].dt.month

# Extract hour and minute separately
df['Departure_Hour'] = df['Departure_24hr'].apply(lambda x: x.hour)
df['Departure_Minute'] = df['Departure_24hr'].apply(lambda x: x.minute)
df['Arrival_Hour'] = df['Arrival_24hr'].apply(lambda x: x.hour)
df['Arrival_Minute'] = df['Arrival_24hr'].apply(lambda x: x.minute)

# Optionally use sine and cosine transformations to encode cyclic patterns
df['Departure_Hour_sin'] = np.sin(2 * np.pi * df['Departure_Hour'] / 24)
df['Departure_Hour_cos'] = np.cos(2 * np.pi * df['Departure_Hour'] / 24)
df['Arrival_Hour_sin'] = np.sin(2 * np.pi * df['Arrival_Hour'] / 24)
df['Arrival_Hour_cos'] = np.cos(2 * np.pi * df['Arrival_Hour'] / 24)

# Drop the original time columns
df.drop(columns=['Departure_24hr', 'Arrival_24hr'], inplace=True)

# One-hot encoding for categorical columns
categorical_columns = ['Airline', 'Source', 'Destination', 'Class']
df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

# Define features and target
features = df.drop(columns=['price in CAD', 'Date'])
target = df['price in CAD']

# Scaling features
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features_scaled, target, test_size=0.2, random_state=42)

# Define the ANN model
ann_model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1)  # Output layer for regression
])

# Compile the ANN model
ann_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the ANN model
history_ann = ann_model.fit(X_train, y_train, epochs=20, batch_size=64, validation_split=0.2, verbose=1)

# Evaluate the ANN model
ann_eval = ann_model.evaluate(X_test, y_test)
print(f'ANN Evaluation: {ann_eval}')

# Predict on the test data
y_pred = ann_model.predict(X_test)

# Compute R^2
r2 = r2_score(y_test, y_pred)
print(f'R^2 score: {r2:.4f}')


Epoch 1/20
8241/8241 ━━━━━━━━━━━━━━━━━━━━ 3s 277us/step - loss: 5188062.0000 - mae: 1542.0938 - val_loss: 1991670.2500 - val_mae: 868.5481
Epoch 2/20
8241/8241 ━━━━━━━━━━━━━━━━━━━━ 2s 270us/step - loss: 2179056.0000 - mae: 944.6491 - val_loss: 1850826.6250 - val_mae: 829.5732
Epoch 3/20
8241/8241 ━━━━━━━━━━━━━━━━━━━━ 2s 272us/step - loss: 2074280.3750 - mae: 919.7256 - val_loss: 1812006.7500 - val_mae: 813.1289
Epoch 4/20
8241/8241 ━━━━━━━━━━━━━━━━━━━━ 2s 270us/step - loss: 2023059.7500 - mae: 907.5956 - val_loss: 1758485.2500 - val_mae: 804.4272
Epoch 5/20
8241/8241 ━━━━━━━━━━━━━━━━━━━━ 2s 274us/step - loss: 2025125.3750 - mae: 903.2421 - val_loss: 1750075.6250 - val_mae: 788.3865
Epoch 6/20
8241/8241 ━━━━━━━━━━━━━━━━━━━━ 2s 269us/step - loss: 1997026.0000 - mae: 895.0446 - val_loss: 1720058.5000 - val_mae: 793.4044
Epoch 7/20
8241/8241 ━━━━━━━━━━━━━━━━━━━━ 2s 273us/step - loss: 1993887.3750 - mae: 890.9796 - val_loss: 1696454.1250 - val_mae: 785.6510
Epoch 8/20
8241/8241 ━━━━━━━━━━━━